In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pandas as pd
import numpy as np

In [3]:
sc = SparkContext( 'local' )
sqlCtx = SQLContext( sc )

22/04/27 16:41:28 WARN Utils: Your hostname, mobile-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.41.128 instead (on interface ens33)
22/04/27 16:41:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/04/27 16:41:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
emp =[('홍길동',1),('이순신',2),
      ('임꺽정',3),('김철수',3),('김철수1',5)]
dept = [('개발',1), ('연구',2),
        ('영업',3),('기획',4) ]

In [4]:
empRdd = sc.parallelize( emp )
empRdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262

In [5]:
empRdd.collect()

[('홍길동', 1), ('이순신', 2), ('임꺽정', 3), ('김철수', 3), ('김철수1', 5)]

### rdd 데이터프레임으로 변환

In [12]:
empDF = empRdd.toDF(['name','deptid']) #분산데이터프레임 파이썬 데이터프레임 아님

In [13]:
empDF

DataFrame[name: string, deptid: bigint]

In [15]:
empDF.printSchema()

root
 |-- name: string (nullable = true)
 |-- deptid: long (nullable = true)



In [14]:
empDF.show()

+-------+------+
|   name|deptid|
+-------+------+
| 홍길동|     1|
| 이순신|     2|
| 임꺽정|     3|
| 김철수|     3|
|김철수1|     5|
+-------+------+



In [17]:
pdf = empDF.toPandas() #판다스 데이터프레임 객체를 반환 (주의요망!)
pdf

,name,deptid
0,홍길동,1
1,이순신,2
2,임꺽정,3
3,김철수,3
4,김철수1,5


In [ ]:
# sql : standard
# plus (function)
# hive sqlg

In [18]:
empDF.createOrReplaceTempView('empt')

In [20]:
sql = 'select * from empt'
sqlDF = sqlCtx.sql( sql)
sqlDF.show() #shift + tab

+-------+------+
|   name|deptid|
+-------+------+
| 홍길동|     1|
| 이순신|     2|
| 임꺽정|     3|
| 김철수|     3|
|김철수1|     5|
+-------+------+



In [35]:
sql = 'select * from empt limit 2'
sqlDF = sqlCtx.sql( sql)
sqlDF.show()

+------+------+
|  name|deptid|
+------+------+
|홍길동|     1|
|이순신|     2|
+------+------+



In [24]:
sql = 'select name from empt'
sqlDF = sqlCtx.sql( sql)
sqlDF.show()

+-------+
|   name|
+-------+
| 홍길동|
| 이순신|
| 임꺽정|
| 김철수|
|김철수1|
+-------+



In [21]:
sql = 'select * from empt where deptid=3'
sqlDF = sqlCtx.sql( sql)
sqlDF.show()

+------+------+
|  name|deptid|
+------+------+
|임꺽정|     3|
|김철수|     3|
+------+------+



In [22]:
sql = "select name,deptid from empt where deptid > 2"
sqlDF = sqlCtx.sql( sql)
sqlDF.show()

+-------+------+
|   name|deptid|
+-------+------+
| 임꺽정|     3|
| 김철수|     3|
|김철수1|     5|
+-------+------+



In [23]:
sql = "select name,deptid from empt where deptid=2 or deptid=3"
sqlDF = sqlCtx.sql( sql)
sqlDF.show()

+------+------+
|  name|deptid|
+------+------+
|이순신|     2|
|임꺽정|     3|
|김철수|     3|
+------+------+



In [25]:
sql = "select name,deptid from empt where deptid in(1,2,3)"
sqlDF = sqlCtx.sql( sql)
sqlDF.show()

+------+------+
|  name|deptid|
+------+------+
|홍길동|     1|
|이순신|     2|
|임꺽정|     3|
|김철수|     3|
+------+------+



In [26]:
sql = "select name,deptid from empt where deptid>=2 and deptid<=4"
sqlDF = sqlCtx.sql( sql)
sqlDF.show()

+------+------+
|  name|deptid|
+------+------+
|이순신|     2|
|임꺽정|     3|
|김철수|     3|
+------+------+



In [27]:
sql = "select name,deptid from empt where deptid between 2 and 4"
sqlDF = sqlCtx.sql( sql)
sqlDF.show()

+------+------+
|  name|deptid|
+------+------+
|이순신|     2|
|임꺽정|     3|
|김철수|     3|
+------+------+



In [28]:
sql = "select name,deptid from empt where name='이순신'"
sqlDF = sqlCtx.sql( sql)
sqlDF.show()

+------+------+
|  name|deptid|
+------+------+
|이순신|     2|
+------+------+



In [29]:
sql = "select name,deptid from empt where name like '%김%'"
sqlDF = sqlCtx.sql( sql)
sqlDF.show()

+-------+------+
|   name|deptid|
+-------+------+
| 김철수|     3|
|김철수1|     5|
+-------+------+



In [30]:
sql = "select name,deptid from empt where name rlike '김'"
sqlDF = sqlCtx.sql( sql)
sqlDF.show()

+-------+------+
|   name|deptid|
+-------+------+
| 김철수|     3|
|김철수1|     5|
+-------+------+



In [32]:
sql = "select name,deptid from empt where name rlike '^김'"
sqlDF = sqlCtx.sql( sql)
sqlDF.show()

+-------+------+
|   name|deptid|
+-------+------+
| 김철수|     3|
|김철수1|     5|
+-------+------+



In [31]:
sql = "select name,deptid from empt where name rlike '수$'"
sqlDF = sqlCtx.sql( sql)
sqlDF.show()

+------+------+
|  name|deptid|
+------+------+
|김철수|     3|
+------+------+



In [34]:
sql = "select name,deptid from empt where name rlike '[김홍]'" # 김 or 홍
sqlDF = sqlCtx.sql( sql)
sqlDF.show()

+-------+------+
|   name|deptid|
+-------+------+
| 홍길동|     1|
| 김철수|     3|
|김철수1|     5|
+-------+------+



In [36]:
sql = "select name,deptid, deptid*2 as deptplus from empt" 
sqlDF = sqlCtx.sql( sql)
sqlDF.show()

+-------+------+--------+
|   name|deptid|deptplus|
+-------+------+--------+
| 홍길동|     1|       2|
| 이순신|     2|       4|
| 임꺽정|     3|       6|
| 김철수|     3|       6|
|김철수1|     5|      10|
+-------+------+--------+



In [38]:
sql = '''select name, deptid,
case
when deptid>4 then '많음'
when deptid>=3 then '보통'
else '적음'
end as deptcon
from empt'''
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+-------+------+-------+
|   name|deptid|deptcon|
+-------+------+-------+
| 홍길동|     1|   적음|
| 이순신|     2|   적음|
| 임꺽정|     3|   보통|
| 김철수|     3|   보통|
|김철수1|     5|   많음|
+-------+------+-------+



### 집계함수
#### 집계( max, min, avg, sum, count) 함수(숫자타입)
###  https://cwiki.apache.org/confluence/display/hive/languagemanual+udf


In [39]:
sql = "select max(deptid) from empt"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+-----------+
|max(deptid)|
+-----------+
|          5|
+-----------+



In [48]:
sql = "select sum(deptid) from empt"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+-----------+
|sum(deptid)|
+-----------+
|         14|
+-----------+



In [40]:
sql = "select avg(deptid) from empt"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+-----------+
|avg(deptid)|
+-----------+
|        2.8|
+-----------+



In [41]:
sql = "select count(*) from empt"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+--------+
|count(1)|
+--------+
|       5|
+--------+



In [42]:
sql = "select stddev_samp(deptid) from empt"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+-----------------------------------+
|stddev_samp(CAST(deptid AS DOUBLE))|
+-----------------------------------+
|                 1.4832396974191326|
+-----------------------------------+



In [43]:
sql = "select length(name) from empt"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+------------+
|length(name)|
+------------+
|           3|
|           3|
|           3|
|           3|
|           4|
+------------+



In [44]:
sql = "select substr(name,2) from empt"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+---------------------------+
|substr(name, 2, 2147483647)|
+---------------------------+
|                       길동|
|                       순신|
|                       꺽정|
|                       철수|
|                      철수1|
+---------------------------+



In [45]:
sql = "select substr(name,2,2) from empt"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+------------------+
|substr(name, 2, 2)|
+------------------+
|              길동|
|              순신|
|              꺽정|
|              철수|
|              철수|
+------------------+



### group by

In [46]:
sql = "select avg(deptid) from empt group by deptid"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+-----------+
|avg(deptid)|
+-----------+
|        5.0|
|        1.0|
|        3.0|
|        2.0|
+-----------+



In [47]:
sql = "select sum(deptid) from empt group by deptid"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+-----------+
|sum(deptid)|
+-----------+
|          5|
|          1|
|          6|
|          2|
+-----------+



In [49]:
sql = "select deptid, avg(deptid),sum(deptid) from empt group by deptid"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+------+-----------+-----------+
|deptid|avg(deptid)|sum(deptid)|
+------+-----------+-----------+
|     5|        5.0|          5|
|     1|        1.0|          1|
|     3|        3.0|          6|
|     2|        2.0|          2|
+------+-----------+-----------+



In [82]:
# name salary 컬럼명을 가지는 데이터프레임을 만드시요.
data1 = ['홍길동,1000','이순신,2000','임꺽정,3000',
         '김철수,4000','이황,5000','이이,6000']

In [83]:
dataRdd = sc.parallelize(data1).map(lambda v : v.split(',')).map(lambda v : (v[0],v[1]))
data = dataRdd.toDF(['name','salary'])
data.show()

+------+------+
|  name|salary|
+------+------+
|홍길동|  1000|
|이순신|  2000|
|임꺽정|  3000|
|김철수|  4000|
|  이황|  5000|
|  이이|  6000|
+------+------+



In [84]:
data.createOrReplaceTempView('sal')

In [85]:
# 1. 급여가 가장높은 name과 salary를 출력하시요
sql = "select  * from sal where order by salary desc limit 1 "
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+----+------+
|name|salary|
+----+------+
|이이|  6000|
+----+------+



In [80]:
sql = 'select name, salary from sal where salary = (select max(salary) from sal) '
sqlDF = sqlCtx.sql(sql)
sqlDF.show()

+----+------+
|name|salary|
+----+------+
|이이|  6000|
+----+------+



In [68]:
# 2. name, salary, tax 를 출력하시요 (tax는 급여에서 세금 3.3을 제한값)
sql = "select name, salary, salary*0.967 as tax from sal"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+------+------+--------+
|  name|salary|     tax|
+------+------+--------+
|홍길동|  1000| 967.000|
|이순신|  2000|1934.000|
|임꺽정|  3000|2901.000|
|김철수|  4000|3868.000|
|  이황|  5000|4835.000|
|  이이|  6000|5802.000|
+------+------+--------+



In [69]:
# 3. name 에 '철'이 포함된  name,salary 를 출력하시요
sql = "select name, salary from sal where name like '%철%'"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+------+------+
|  name|salary|
+------+------+
|김철수|  4000|
+------+------+



In [75]:
# 4. salary top 5 인 name, salary 출력하시요
sql = "select name, salary from sal order by salary desc limit 5"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+------+------+
|  name|salary|
+------+------+
|  이이|  6000|
|  이황|  5000|
|김철수|  4000|
|임꺽정|  3000|
|이순신|  2000|
+------+------+



In [71]:
# 5. salary  2000  과 4000 사이의 데이터를 출력하시요
sql = "select * from sal where salary between 2000 and 4000"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+------+------+
|  name|salary|
+------+------+
|이순신|  2000|
|임꺽정|  3000|
|김철수|  4000|
+------+------+



## 과제

In [4]:
cctvDF = sqlCtx.read.csv('cctv.csv', header=True, inferSchema=True) #False면 string인데 True면타입 자동으로 변경
cctvDF.show()

+--------+----+-------------+------+------+------+
|  기관명|소계|2013년도 이전|2014년|2015년|2016년|
+--------+----+-------------+------+------+------+
|  강남구|2780|         1292|   430|   584|   932|
|  강동구| 773|          379|    99|   155|   377|
|  강북구| 748|          369|   120|   138|   204|
|  강서구| 884|          388|   258|   184|    81|
|  관악구|1496|          846|   260|   390|   613|
|  광진구| 707|          573|    78|    53|   174|
|  구로구|1561|         1142|   173|   246|   323|
|  금천구|1015|          674|    51|   269|   354|
|  노원구|1265|          542|    57|   451|   516|
|  도봉구| 485|          238|   159|    42|   386|
|동대문구|1294|         1070|    23|   198|   579|
|  동작구|1091|          544|   341|   103|   314|
|  마포구| 574|          314|   118|   169|   379|
|서대문구| 962|          844|    50|    68|   292|
|  서초구|1930|         1406|   157|   336|   398|
|  성동구|1062|          730|    91|   241|   265|
|  성북구|1464|         1009|    78|   360|   204|
|  송파구| 618|          529|    21|    68|   46

In [5]:
cctvDF.printSchema()

root
 |-- 기관명: string (nullable = true)
 |-- 소계: integer (nullable = true)
 |-- 2013년도 이전: integer (nullable = true)
 |-- 2014년: integer (nullable = true)
 |-- 2015년: integer (nullable = true)
 |-- 2016년: integer (nullable = true)



In [6]:
cctvDF.createOrReplaceTempView('cctv')

In [23]:
# 1. 기관명이 '강'글자가 포함된  데이터를 가져오시요
sql = "select * from cctv where `기관명` rlike '강'"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+------+----+-------------+------+------+------+
|기관명|소계|2013년도 이전|2014년|2015년|2016년|
+------+----+-------------+------+------+------+
|강남구|2780|         1292|   430|   584|   932|
|강동구| 773|          379|    99|   155|   377|
|강북구| 748|          369|   120|   138|   204|
|강서구| 884|          388|   258|   184|    81|
+------+----+-------------+------+------+------+



In [24]:
# 2. 2015년도 cctv 총합을 구하시요.
sql = "select sum(`2015년`) from cctv"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+-----------+
|sum(2015년)|
+-----------+
|       5130|
+-----------+



In [31]:
# 3. 증가율이 가장높은 top 5를 보여주시요
# 증가율:  (14+15+16)/(2013년이전 )
sql = "select `기관명`, (`2014년`+`2015년`+`2016년`)/`2013년도 이전` as `증가율`  from cctv order by `증가율` desc limit 5"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()


+------+------------------+
|기관명|            증가율|
+------+------------------+
|종로구|2.4892241379310347|
|도봉구| 2.466386554621849|
|마포구| 2.121019108280255|
|노원구|1.8892988929889298|
|강동구|1.6649076517150396|
+------+------------------+



In [99]:
# 4. 
# 기관명에서 마지막 '구'라는 글자를 제한 
# 글자를 출력하시요(예)
# 기관명
# ==========
# 강남
# 강동
# 강북....

sql = "select if(length(`기관명`) = 3 , substr(`기관명`,1,2) , substr(`기관명`,1,3) ) as `위치` from cctv"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+------+
|  위치|
+------+
|  강남|
|  강동|
|  강북|
|  강서|
|  관악|
|  광진|
|  구로|
|  금천|
|  노원|
|  도봉|
|동대문|
|  동작|
|  마포|
|서대문|
|  서초|
|  성동|
|  성북|
|  송파|
|  양천|
|영등포|
+------+
only showing top 20 rows



In [57]:
# 5. cctv가 가장많은 기관명과 갯수를 구하시요 .
sql = "select `기관명`, `2014년`+`2015년`+`2016년`+`2013년도 이전` as `총개수` from cctv order by `총개수` desc limit 1"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+------+------+
|기관명|총개수|
+------+------+
|강남구|  3238|
+------+------+



In [100]:
# 6.  cctv총갯수가 1500개 이상이면 '매우많음' 1000개이상이면 '많음' 
# 나머지 보통으로 표시하시요
sql = '''select *,`2014년`+`2015년`+`2016년`+`2013년도 이전` as `총개수`,
case
when `2014년`+`2015년`+`2016년`+`2013년도 이전` >=1500 then '매우많음'
when `2014년`+`2015년`+`2016년`+`2013년도 이전` >=1000 then '많음'
else '보통'
end as `표시`
from cctv'''
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

+--------+----+-------------+------+------+------+------+--------+
|  기관명|소계|2013년도 이전|2014년|2015년|2016년|총개수|    표시|
+--------+----+-------------+------+------+------+------+--------+
|  강남구|2780|         1292|   430|   584|   932|  3238|매우많음|
|  강동구| 773|          379|    99|   155|   377|  1010|    많음|
|  강북구| 748|          369|   120|   138|   204|   831|    보통|
|  강서구| 884|          388|   258|   184|    81|   911|    보통|
|  관악구|1496|          846|   260|   390|   613|  2109|매우많음|
|  광진구| 707|          573|    78|    53|   174|   878|    보통|
|  구로구|1561|         1142|   173|   246|   323|  1884|매우많음|
|  금천구|1015|          674|    51|   269|   354|  1348|    많음|
|  노원구|1265|          542|    57|   451|   516|  1566|매우많음|
|  도봉구| 485|          238|   159|    42|   386|   825|    보통|
|동대문구|1294|         1070|    23|   198|   579|  1870|매우많음|
|  동작구|1091|          544|   341|   103|   314|  1302|    많음|
|  마포구| 574|          314|   118|   169|   379|   980|    보통|
|서대문구| 962|          

In [ ]:
# 7. 2014년도 2015년도 cctv데이터에 대한 바차트를 그리시요(x축 기관명임)

In [103]:
sql = "select * ,REPLICATE('▒', `기관명` )  [Bar] from cctv"
sqlDF = sqlCtx.sql( sql )
sqlDF.show()

AnalysisException: Undefined function: 'REPLICATE'. This function is neither a registered temporary function nor a permanent function registered in the database 'default'.; line 1 pos 10